# Chess Game Analysis Notebook

## ⚡ Quick Start
1. **Run the setup cell below first** - Validates environment and reloads latest code
2. Configure your analysis parameters (username, date range)
3. Fetch game data from Chess.com
4. Run comprehensive analysis
5. View all 8 analysis sections

---

## 🔧 Setup & Environment Check
Run this cell whenever you:
- Start a fresh session
- Make changes to service files (analytics_service.py, chess_service.py, etc.)
- See import errors or stale data

In [50]:
# ============================================================
# SETUP VALIDATION & MODULE RELOAD
# ============================================================
# Run this cell first to ensure environment is properly configured
# and modules are reloaded with latest changes

import sys
import os
from pathlib import Path

print("=" * 60)
print("🔧 SETUP VALIDATION")
print("=" * 60)

# 1. Check Python version
print(f"\n✓ Python Version: {sys.version.split()[0]}")

# 2. Check working directory
project_root = str(Path(__file__).parent.parent.parent) if '__file__' in globals() else os.getcwd()
print(f"✓ Project Root: {project_root}")

# 3. Add project root to path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print("✓ Added project root to sys.path")

# 4. Check required packages
print("\n📦 Checking Required Packages...")
required_packages = {
    'pandas': 'pandas',
    'matplotlib': 'matplotlib', 
    'plotly': 'plotly',
    'chess': 'chess',
    'openai': 'openai'
}

missing_packages = []
for import_name, package_name in required_packages.items():
    try:
        __import__(import_name)
        print(f"  ✓ {package_name}")
    except ImportError:
        print(f"  ✗ {package_name} - NOT INSTALLED")
        missing_packages.append(package_name)

if missing_packages:
    print(f"\n⚠️  Missing packages: {', '.join(missing_packages)}")
    print("   Run the package installation cell below to install them.")
else:
    print("\n✓ All required packages are installed")

# 5. Reload modules to get latest code changes
print("\n🔄 Reloading Modules...")
modules_to_reload = [
    'app.services.analytics_service',
    'app.services.chess_service', 
    'app.services.chess_advisor_service',
    'app.services.mistake_analysis_service',
    'app.utils.timezone_utils',
    'app.utils.validators',
    'app.utils.cache'
]

reloaded_count = 0
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]
        reloaded_count += 1

if reloaded_count > 0:
    print(f"  ✓ Cleared {reloaded_count} cached modules")
else:
    print("  ℹ️  No cached modules to reload")

# 6. Import and instantiate services
print("\n🚀 Initializing Services...")
try:
    from app.services.chess_service import ChessService
    from app.services.analytics_service import AnalyticsService
    
    chess_service = ChessService()
    analytics_service = AnalyticsService()
    
    print("  ✓ ChessService initialized")
    print("  ✓ AnalyticsService initialized")
    
    # Make services available globally
    globals()['chess_service'] = chess_service
    globals()['analytics_service'] = analytics_service
    
except ImportError as e:
    print(f"  ✗ Error importing services: {e}")
    print("  ⚠️  Make sure you're running from the project root directory")
except Exception as e:
    print(f"  ✗ Error initializing services: {e}")

print("\n" + "=" * 60)
print("✅ SETUP COMPLETE")
print("=" * 60)
print("\n💡 Tips:")
print("   • Re-run this cell after making changes to service files")
print("   • This ensures you always have the latest code loaded")
print("   • If you see import errors, check your working directory")

🔧 SETUP VALIDATION

✓ Python Version: 3.12.4
✓ Project Root: c:\anaconda_backup\Project\chesstic_v2\.github\analysis

📦 Checking Required Packages...
  ✓ pandas
  ✓ matplotlib
  ✓ plotly
  ✓ chess
  ✓ openai

✓ All required packages are installed

🔄 Reloading Modules...
  ✓ Cleared 6 cached modules

🚀 Initializing Services...
  ✓ ChessService initialized
  ✓ AnalyticsService initialized

✅ SETUP COMPLETE

💡 Tips:
   • Re-run this cell after making changes to service files
   • This ensures you always have the latest code loaded
   • If you see import errors, check your working directory


In [ ]:
# Chess Game Data Analysis

This notebook pulls chess game data from Chess.com API and displays comprehensive analytics across all sections.

## Sections Covered:
1. **Overall Performance** - Win rate, rating trends
2. **Color Performance** - White vs Black statistics
3. **ELO Progression** - Rating changes over time
4. **Termination Analysis** - How games end (wins/losses)
5. **Opening Performance** - Best and worst openings
6. **Opponent Strength** - Performance against different ratings
7. **Time of Day** - Performance by time periods
8. **Mistake Analysis** - Game stage mistakes (early/middle/endgame)
9. **AI Coaching Advice** - Personalized recommendations

## 🚀 Quick Setup - Run This First!

This cell checks your environment and installs required packages automatically.
**Run this cell first** every time you open the notebook.

In [3]:
# Quick Setup: Check and install required packages
import sys
import subprocess

print("="*60)
print("🔧 NOTEBOOK SETUP CHECK")
print("="*60)

# Check Python environment
print(f"\n✓ Python: {sys.version.split()[0]}")
print(f"✓ Environment: {sys.executable}")

# Required packages for this notebook
required_packages = {
    'pandas': 'pandas',
    'matplotlib': 'matplotlib', 
    'plotly': 'plotly',
    'chess': 'chess',
    'openai': 'openai'
}

# Check which packages are missing
missing_packages = []
for import_name, package_name in required_packages.items():
    try:
        __import__(import_name)
        print(f"✓ {package_name} is installed")
    except ImportError:
        print(f"✗ {package_name} is NOT installed")
        missing_packages.append(package_name)

# Install missing packages if any
if missing_packages:
    print(f"\n📦 Installing missing packages: {', '.join(missing_packages)}")
    print("⏳ This may take a minute...")
    
    for package in missing_packages:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
    
    print("✓ All packages installed successfully!")
else:
    print("\n✓ All required packages are already installed!")

print("\n" + "="*60)
print("✅ SETUP COMPLETE - Ready to run analysis!")
print("="*60)
print("\n💡 Tip: Run cells in order from top to bottom.")

🔧 NOTEBOOK SETUP CHECK

✓ Python: 3.12.4
✓ Environment: c:\anaconda_backup\Project\chesstic_v2\.venv\Scripts\python.exe
✓ pandas is installed
✓ matplotlib is installed
✓ plotly is installed
✓ chess is installed
✓ openai is installed

✓ All required packages are already installed!

✅ SETUP COMPLETE - Ready to run analysis!

💡 Tip: Run cells in order from top to bottom.


In [1]:
# Import required libraries
import sys
import os
from datetime import datetime, timedelta
import pandas as pd
import json

# Add project root to path to import project modules
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, project_root)

from app.services.chess_service import ChessService
from app.services.analytics_service import AnalyticsService

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
# Configuration - Update these values
USERNAME = 'jay_fh'  # Chess.com username to analyze
TIMEZONE = 'America/New_York'  # User timezone

# Calculate date range for last 2 weeks
END_DATE_DT = datetime.now()
START_DATE_DT = END_DATE_DT - timedelta(days=14)
START_DATE = START_DATE_DT.strftime('%Y-%m-%d')
END_DATE = END_DATE_DT.strftime('%Y-%m-%d')

# Initialize services
chess_service = ChessService()
analytics_service = AnalyticsService(
    stockfish_path='stockfish',  # Update path if needed
    engine_depth=12,
    engine_enabled=True,  # Set to False to skip mistake analysis
    openai_api_key='',    # Add OpenAI API key for AI advice
    openai_model='gpt-4o-mini'
)

print(f"Configuration:")
print(f"  Username: {USERNAME}")
print(f"  Date Range: {START_DATE} to {END_DATE} (Last 2 weeks)")
print(f"  Timezone: {TIMEZONE}")
print(f"✓ Services initialized")

Configuration:
  Username: jay_fh
  Date Range: 2026-01-27 to 2026-02-10 (Last 2 weeks)
  Timezone: America/New_York
✓ Services initialized


## Step 1: Fetch Chess Games Data

Pulling games from Chess.com API for the specified date range.

In [21]:
START_DATE
END_DATE

'2026-02-07'

## Diagnostic: Check Date Range and Available Games

In [4]:
# Diagnostic: Check date range and API response
print("="*60)
print("DATE RANGE DIAGNOSTIC")
print("="*60)
print(f"START_DATE: {START_DATE}")
print(f"END_DATE: {END_DATE}")
print(f"Current date: {datetime.now().strftime('%Y-%m-%d')}")
print(f"Days in range: {(datetime.strptime(END_DATE, '%Y-%m-%d') - datetime.strptime(START_DATE, '%Y-%m-%d')).days}")

# Check what Chess.com API returns for February 2026
print(f"\n{'='*60}")
print("CHECKING FEBRUARY 2026 GAMES FROM API")
print("="*60)
try:
    feb_games = chess_service.get_games_by_month(USERNAME, 2026, 2)
    print(f"Total games in Feb 2026 from API: {len(feb_games)}")
    
    if feb_games:
        # Show dates of February games
        feb_dates = [datetime.fromtimestamp(g.get('end_time', 0)) for g in feb_games]
        print(f"First game: {min(feb_dates).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Last game: {max(feb_dates).strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Check games within our date range
        in_range = [d for d in feb_dates if datetime.strptime(START_DATE, '%Y-%m-%d') <= d <= datetime.strptime(END_DATE, '%Y-%m-%d')]
        print(f"\nGames in Feb within date range ({START_DATE} to {END_DATE}): {len(in_range)}")
        
        if in_range:
            print("Dates of games in range:")
            for date in sorted(in_range):
                print(f"  - {date.strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        print("No games found in February 2026")
except Exception as e:
    print(f"Error fetching February games: {str(e)}")

print(f"\n{'='*60}")
print("CONCLUSION")
print("="*60)
print("If no games appear from Feb 2-7, it means the user hasn't")
print("played any games during that period on Chess.com.")

DATE RANGE DIAGNOSTIC
START_DATE: 2026-01-27
END_DATE: 2026-02-10
Current date: 2026-02-10
Days in range: 14

CHECKING FEBRUARY 2026 GAMES FROM API
Total games in Feb 2026 from API: 65
First game: 2026-02-02 14:06:42
Last game: 2026-02-10 22:38:56

Games in Feb within date range (2026-01-27 to 2026-02-10): 52
Dates of games in range:
  - 2026-02-02 14:06:42
  - 2026-02-02 17:16:21
  - 2026-02-02 17:23:50
  - 2026-02-02 17:34:21
  - 2026-02-02 17:44:33
  - 2026-02-03 17:15:41
  - 2026-02-03 23:55:20
  - 2026-02-04 00:07:02
  - 2026-02-04 00:18:05
  - 2026-02-04 13:50:06
  - 2026-02-05 00:06:57
  - 2026-02-05 11:46:06
  - 2026-02-05 11:50:27
  - 2026-02-05 12:11:25
  - 2026-02-05 12:14:33
  - 2026-02-05 12:18:49
  - 2026-02-05 12:28:10
  - 2026-02-05 12:48:41
  - 2026-02-05 12:58:27
  - 2026-02-05 13:05:59
  - 2026-02-05 16:41:02
  - 2026-02-05 23:11:43
  - 2026-02-05 23:22:49
  - 2026-02-06 09:14:51
  - 2026-02-06 12:12:51
  - 2026-02-06 12:20:19
  - 2026-02-06 13:10:24
  - 2026-02-06 1

In [7]:
# Fetch games from Chess.com API
print(f"Fetching games for {USERNAME} from {START_DATE} to {END_DATE}...")

start = datetime.strptime(START_DATE, '%Y-%m-%d')
end = datetime.strptime(END_DATE, '%Y-%m-%d')

all_games = []
current = start

# Fetch games for each month in the range
while current <= end:
    try:
        games = chess_service.get_games_by_month(USERNAME, current.year, current.month)
        
        # Filter games by date range
        filtered_games = []
        for game in games:
            game_date = datetime.fromtimestamp(game.get('end_time', 0))
            if start <= game_date <= end:
                filtered_games.append(game)
        
        all_games.extend(filtered_games)
        print(f"  ✓ {current.strftime('%Y-%m')}: {len(filtered_games)} games")
    except Exception as e:
        print(f"  ✗ {current.strftime('%Y-%m')}: No games or error - {str(e)}")
    
    # Move to next month (set day to 1 to ensure we check all months in range)
    if current.month == 12:
        current = current.replace(year=current.year + 1, month=1, day=1)
    else:
        current = current.replace(month=current.month + 1, day=1)

print(f"\n✓ Total games fetched: {len(all_games)}")

# Display sample game data
if all_games:
    print("\nSample game data (first game):")
    sample = all_games[0]
    print(f"  URL: {sample.get('url', 'N/A')}")
    print(f"  Time Class: {sample.get('time_class', 'N/A')}")
    print(f"  White: {sample.get('white', {}).get('username', 'N/A')} ({sample.get('white', {}).get('rating', 'N/A')})")
    print(f"  Black: {sample.get('black', {}).get('username', 'N/A')} ({sample.get('black', {}).get('rating', 'N/A')})")
    print(f"  Result: {sample.get('white', {}).get('result', 'N/A')} - {sample.get('black', {}).get('result', 'N/A')}")

Fetching games for jay_fh from 2026-01-27 to 2026-02-10...
  ✓ 2026-01: 30 games
  ✓ 2026-02: 52 games

✓ Total games fetched: 82

Sample game data (first game):
  URL: https://www.chess.com/game/live/148519596048
  Time Class: blitz
  White: Ossichesss (1772)
  Black: jay_fh (1808)
  Result: timeout - win


## Step 1.1: Convert Games to DataFrame

Creating a structured DataFrame from the fetched games for easier analysis and data manipulation.

In [8]:
# Convert games data to DataFrame for easier analysis
if all_games:
    # Extract key information from each game
    games_data = []
    for game in all_games:
        # Determine player color and result
        is_white = game.get('white', {}).get('username', '').lower() == USERNAME.lower()
        player_color = 'white' if is_white else 'black'
        opponent_color = 'black' if is_white else 'white'
        
        # Get player and opponent data
        player_data = game.get(player_color, {})
        opponent_data = game.get(opponent_color, {})
        
        # Parse game data
        game_info = {
            'url': game.get('url', ''),
            'game_id': game.get('url', '').split('/')[-1] if game.get('url') else '',
            'date': datetime.fromtimestamp(game.get('end_time', 0)),
            'time_class': game.get('time_class', ''),
            'rules': game.get('rules', ''),
            'player_color': player_color,
            'player_rating': player_data.get('rating', 0),
            'player_result': player_data.get('result', ''),
            'opponent_username': opponent_data.get('username', ''),
            'opponent_rating': opponent_data.get('rating', 0),
            'opponent_result': opponent_data.get('result', ''),
            'rating_diff': player_data.get('rating', 0) - opponent_data.get('rating', 0),
            'pgn': game.get('pgn', ''),
            'eco': game.get('eco', ''),
            'opening': game.get('opening', ''),
            'time_control': game.get('time_control', '')
        }
        games_data.append(game_info)
    
    # Create DataFrame
    df_games = pd.DataFrame(games_data)
    
    # Add derived columns
    df_games['result'] = df_games['player_result'].apply(
        lambda x: 'Win' if x == 'win' else ('Loss' if x in ['checkmated', 'resigned', 'timeout', 'abandoned'] else 'Draw')
    )
    df_games['day_of_week'] = df_games['date'].dt.day_name()
    df_games['hour'] = df_games['date'].dt.hour
    
    print(f"✓ DataFrame created with {len(df_games)} games")
    print(f"\nDataFrame shape: {df_games.shape}")
    print(f"Columns: {list(df_games.columns)}")
    print("\n" + "="*60)
    print("Sample Data (first 3 games):")
    print("="*60)
    
    # Display key columns
    display_cols = ['date', 'player_color', 'result', 'player_rating', 'opponent_rating', 'rating_diff', 'time_class']
    print(df_games[display_cols].head(3).to_string(index=False))
    
    print("\n" + "="*60)
    print("Summary Statistics:")
    print("="*60)
    print(f"Total Games: {len(df_games)}")
    print(f"Wins: {(df_games['result'] == 'Win').sum()}")
    print(f"Losses: {(df_games['result'] == 'Loss').sum()}")
    print(f"Draws: {(df_games['result'] == 'Draw').sum()}")
    print(f"Win Rate: {(df_games['result'] == 'Win').sum() / len(df_games) * 100:.1f}%")
    print(f"\nDate Range: {df_games['date'].min().strftime('%Y-%m-%d')} to {df_games['date'].max().strftime('%Y-%m-%d')}")
    print(f"Time Classes: {df_games['time_class'].value_counts().to_dict()}")
else:
    print("⚠️ No games to convert to DataFrame")
    df_games = pd.DataFrame()

✓ DataFrame created with 82 games

DataFrame shape: (82, 19)
Columns: ['url', 'game_id', 'date', 'time_class', 'rules', 'player_color', 'player_rating', 'player_result', 'opponent_username', 'opponent_rating', 'opponent_result', 'rating_diff', 'pgn', 'eco', 'opening', 'time_control', 'result', 'day_of_week', 'hour']

Sample Data (first 3 games):
               date player_color result  player_rating  opponent_rating  rating_diff time_class
2026-01-27 00:01:34        black    Win           1808             1772           36      blitz
2026-01-27 00:07:28        white    Win           1816             1819           -3      blitz
2026-01-27 00:13:31        black    Win           1823             1781           42      blitz

Summary Statistics:
Total Games: 82
Wins: 43
Losses: 38
Draws: 1
Win Rate: 52.4%

Date Range: 2026-01-27 to 2026-02-09
Time Classes: {'blitz': 82}


## Step 2: Perform Comprehensive Analysis

Running detailed analysis across all sections (milestones 1-9).

In [51]:
# Perform comprehensive analysis
print("Running comprehensive analysis...")
print("Note: This may take a few minutes if mistake analysis is enabled.\n")

date_range_str = f"{START_DATE} to {END_DATE}"

analysis_results = analytics_service.analyze_detailed(
    games=all_games,
    username=USERNAME,
    timezone=TIMEZONE,
    include_mistake_analysis=True,  # Milestone 8
    include_ai_advice=False,  # Milestone 9 - requires OpenAI API key
    date_range=date_range_str
)

print("✓ Analysis complete!")
print(f"\nTotal games analyzed: {analysis_results.get('total_games', 0)}")

Running comprehensive analysis...
Note: This may take a few minutes if mistake analysis is enabled.



Failed to start Stockfish engine: 
Engine not available, skipping mistake analysis


✓ Analysis complete!

Total games analyzed: 82


## Section 1: Overall Performance

General performance metrics including win rate and rating trends.

In [43]:
# Display Overall Performance
overall = analysis_results['sections']['overall_performance']

print("=" * 60)
print("OVERALL PERFORMANCE")
print("=" * 60)
print(f"Win Rate: {overall.get('win_rate', 0):.1f}%")
print(f"Rating Change: {overall.get('rating_change', 0):+.0f}")
print(f"Rating Trend: {overall.get('rating_trend', 'N/A')}")
print(f"Average Rating: {overall.get('avg_rating', 0):.0f}")
print(f"\nTotal Record:")
print(f"  Wins: {overall.get('total', {}).get('wins', 0)}")
print(f"  Losses: {overall.get('total', {}).get('losses', 0)}")
print(f"  Draws: {overall.get('total', {}).get('draws', 0)}")

# Display daily stats if available
daily_stats = overall.get('daily_stats', [])
if daily_stats:
    print(f"\nDaily Statistics: {len(daily_stats)} days")
    df_daily = pd.DataFrame(daily_stats)
    print("\nSample Daily Stats (first 5 days):")
    # print(df_daily.head())
    print(df_daily)
    

OVERALL PERFORMANCE
Win Rate: 52.4%
Rating Change: +22
Rating Trend: Improving
Average Rating: 1829

Total Record:
  Wins: 43
  Losses: 38
  Draws: 1

Daily Statistics: 14 days

Sample Daily Stats (first 5 days):
          date  wins  losses  draws
0   2026-01-26     4       1      0
1   2026-01-27     1       5      1
2   2026-01-28     2       0      0
3   2026-01-29     4       1      0
4   2026-01-30     4       5      0
5   2026-01-31     0       2      0
6   2026-02-02     4       1      0
7   2026-02-03     2       2      0
8   2026-02-04     3       1      0
9   2026-02-05     4       7      0
10  2026-02-06     7       4      0
11  2026-02-07     4       2      0
12  2026-02-08     3       3      0
13  2026-02-09     1       4      0


## Section 2: Color Performance

Performance comparison between playing White vs Black pieces.

In [18]:
# Display Color Performance
color_perf = analysis_results['sections']['color_performance']

print("=" * 60)
print("COLOR PERFORMANCE")
print("=" * 60)

# White performance
white = color_perf.get('white', {})
print("\n🔲 WHITE PIECES:")
print(f"  Win Rate: {white.get('win_rate', 0):.1f}%")
print(f"  Games: {white.get('total', {}).get('wins', 0)}W - {white.get('total', {}).get('losses', 0)}L - {white.get('total', {}).get('draws', 0)}D")

# Black performance
black = color_perf.get('black', {})
print("\n⬛ BLACK PIECES:")
print(f"  Win Rate: {black.get('win_rate', 0):.1f}%")
print(f"  Games: {black.get('total', {}).get('wins', 0)}W - {black.get('total', {}).get('losses', 0)}L - {black.get('total', {}).get('draws', 0)}D")

# Comparison
diff = white.get('win_rate', 0) - black.get('win_rate', 0)
stronger = "White" if diff > 0 else "Black"
print(f"\n📊 Stronger Color: {stronger} ({abs(diff):.1f}% difference)")

COLOR PERFORMANCE

🔲 WHITE PIECES:
  Win Rate: 53.7%
  Games: 22W - 19L - 0D

⬛ BLACK PIECES:
  Win Rate: 51.2%
  Games: 21W - 19L - 1D

📊 Stronger Color: White (2.4% difference)


## Section 3: ELO Progression

Rating changes over the analyzed period.

In [35]:
# Display ELO Progression
elo_prog = analysis_results['sections']['elo_progression']

print("=" * 60)
print("ELO PROGRESSION")
print("=" * 60)
print(f"Rating Change: {elo_prog.get('rating_change', 0):+.0f}")
print(f"Starting Rating: {elo_prog.get('start_rating', 0):.0f}")
print(f"Ending Rating: {elo_prog.get('end_rating', 0):.0f}")
print(f"Peak Rating: {elo_prog.get('peak_rating', 0):.0f}")
print(f"Lowest Rating: {elo_prog.get('lowest_rating', 0):.0f}")

# Display data points
data_points = elo_prog.get('data_points', [])
if data_points:
    print(f"\nRating Data Points: {len(data_points)}")
    df_elo = pd.DataFrame(data_points)
    print("\nSample ELO Data (first 5 games):")
    print(df_elo.head())

ELO PROGRESSION
Rating Change: +8
Starting Rating: 1822
Ending Rating: 1830
Peak Rating: 1867
Lowest Rating: 1788

Rating Data Points: 14

Sample ELO Data (first 5 games):
         date  rating
0  2026-01-26    1822
1  2026-01-27    1788
2  2026-01-28    1804
3  2026-01-29    1828
4  2026-01-30    1818


## Section 4: Termination Analysis

How games end - both wins and losses.

In [36]:
# Display Termination Analysis
term_wins = analysis_results['sections']['termination_wins']
term_losses = analysis_results['sections']['termination_losses']

print("=" * 60)
print("TERMINATION ANALYSIS")
print("=" * 60)

# Wins
print("\n✅ HOW YOU WIN:")
print(f"  Total Wins: {term_wins.get('total_wins', 0)}")
wins_breakdown = term_wins.get('breakdown', {})
for method, count in sorted(wins_breakdown.items(), key=lambda x: x[1], reverse=True):
    pct = (count / term_wins.get('total_wins', 1)) * 100
    print(f"    {method}: {count} ({pct:.1f}%)")

# Losses
print("\n❌ HOW YOU LOSE:")
print(f"  Total Losses: {term_losses.get('total_losses', 0)}")
losses_breakdown = term_losses.get('breakdown', {})
for method, count in sorted(losses_breakdown.items(), key=lambda x: x[1], reverse=True):
    pct = (count / term_losses.get('total_losses', 1)) * 100
    print(f"    {method}: {count} ({pct:.1f}%)")

TERMINATION ANALYSIS

✅ HOW YOU WIN:
  Total Wins: 43
    resignation: 25 (58.1%)
    checkmate: 9 (20.9%)
    timeout: 7 (16.3%)
    abandoned: 2 (4.7%)

❌ HOW YOU LOSE:
  Total Losses: 38
    resignation: 25 (65.8%)
    timeout: 6 (15.8%)
    checkmate: 5 (13.2%)
    abandoned: 2 (5.3%)


## Section 5: Opening Performance

Best and worst performing chess openings.

In [52]:
# Display Opening Performance
from urllib.parse import quote

openings = analysis_results['sections']['opening_performance']

print("=" * 60)
print("OPENING PERFORMANCE")
print("=" * 60)

# White openings
white_openings = openings.get('white', {})
print("\n♔ PLAYING AS WHITE")
print("-" * 60)

# Best white openings
best_white = white_openings.get('best_openings', [])
print("\n🏆 BEST OPENINGS (White):")
if best_white:
    for i, opening in enumerate(best_white[:5], 1):
        print(f"\n  {i}. {opening.get('opening', 'N/A')}")
        print(f"     Win Rate: {opening.get('win_rate', 0):.1f}% | Games: {opening.get('games', 0)} ({opening.get('wins', 0)}W-{opening.get('losses', 0)}L-{opening.get('draws', 0)}D)")
        moves = opening.get('first_moves', '')
        if moves:
            print(f"     Moves: {moves}")
        fen = opening.get('fen', '')
        if fen:
            lichess_url = f"https://lichess.org/editor/{quote(fen)}"
            print(f"     📊 View Position: {lichess_url}")
else:
    print("  No data available")

# Worst white openings
worst_white = white_openings.get('worst_openings', [])
print("\n📉 WORST OPENINGS (White):")
if worst_white:
    for i, opening in enumerate(worst_white[:5], 1):
        print(f"\n  {i}. {opening.get('opening', 'N/A')}")
        print(f"     Win Rate: {opening.get('win_rate', 0):.1f}% | Games: {opening.get('games', 0)} ({opening.get('wins', 0)}W-{opening.get('losses', 0)}L-{opening.get('draws', 0)}D)")
        moves = opening.get('first_moves', '')
        if moves:
            print(f"     Moves: {moves}")
        fen = opening.get('fen', '')
        if fen:
            lichess_url = f"https://lichess.org/editor/{quote(fen)}"
            print(f"     📊 View Position: {lichess_url}")
else:
    print("  No data available")

# Black openings
black_openings = openings.get('black', {})
print("\n\n♚ PLAYING AS BLACK")
print("-" * 60)

# Best black openings
best_black = black_openings.get('best_openings', [])
print("\n🏆 BEST OPENINGS (Black):")
if best_black:
    for i, opening in enumerate(best_black[:5], 1):
        print(f"\n  {i}. {opening.get('opening', 'N/A')}")
        print(f"     Win Rate: {opening.get('win_rate', 0):.1f}% | Games: {opening.get('games', 0)} ({opening.get('wins', 0)}W-{opening.get('losses', 0)}L-{opening.get('draws', 0)}D)")
        moves = opening.get('first_moves', '')
        if moves:
            print(f"     Moves: {moves}")
        fen = opening.get('fen', '')
        if fen:
            lichess_url = f"https://lichess.org/editor/{quote(fen)}"
            print(f"     📊 View Position: {lichess_url}")
else:
    print("  No data available")

# Worst black openings
worst_black = black_openings.get('worst_openings', [])
print("\n📉 WORST OPENINGS (Black):")
if worst_black:
    for i, opening in enumerate(worst_black[:5], 1):
        print(f"\n  {i}. {opening.get('opening', 'N/A')}")
        print(f"     Win Rate: {opening.get('win_rate', 0):.1f}% | Games: {opening.get('games', 0)} ({opening.get('wins', 0)}W-{opening.get('losses', 0)}L-{opening.get('draws', 0)}D)")
        moves = opening.get('first_moves', '')
        if moves:
            print(f"     Moves: {moves}")
        fen = opening.get('fen', '')
        if fen:
            lichess_url = f"https://lichess.org/editor/{quote(fen)}"
            print(f"     📊 View Position: {lichess_url}")
else:
    print("  No data available")


OPENING PERFORMANCE

♔ PLAYING AS WHITE
------------------------------------------------------------

🏆 BEST OPENINGS (White):

  1. Pirc Defense
     Win Rate: 75.0% | Games: 4 (3W-1L-0D)
     Moves: 1. d4 d6 2. e3 e6 3. c4 Bd7 4. Nc3 c6 5. Nf3 g5 6. h3 g4
     📊 View Position: https://lichess.org/editor/rn1qkbnr/pp1b1p1p/2ppp3/8/2PP2p1/2N1PN1P/PP3PP1/R1BQKB1R%20w%20KQkq%20-%200%207

  2. Queen's Pawn Game
     Win Rate: 71.4% | Games: 14 (10W-4L-0D)
     Moves: 1. d4 d5 2. e3 f5 3. c4 e6 4. Nc3 c6 5. Nf3 Nf6 6. a3 Bd6
     📊 View Position: https://lichess.org/editor/rnbqk2r/pp4pp/2pbpn2/3p1p2/2PP4/P1N1PN2/1P3PPP/R1BQKB1R%20w%20KQkq%20-%201%207

  3. Scandinavian Defense
     Win Rate: 50.0% | Games: 2 (1W-1L-0D)
     Moves: 1. e4 d5 2. d4 dxe4 3. Nc3 e5 4. d5 Bf5 5. Be3 Nf6 6. h3 Bb4
     📊 View Position: https://lichess.org/editor/rn1qk2r/ppp2ppp/5n2/3Ppb2/1b2p3/2N1B2P/PPP2PP1/R2QKBNR%20w%20KQkq%20-%201%207

  4. French Defense
     Win Rate: 50.0% | Games: 2 (1W-1L-0D)
     Moves: 

## Section 6: Opponent Strength Analysis

Performance against different rating ranges.

In [38]:
# Display Opponent Strength Analysis
opponent = analysis_results['sections']['opponent_strength']

print("=" * 60)
print("OPPONENT STRENGTH ANALYSIS")
print("=" * 60)
print(f"\nAverage Opponent Rating: {opponent.get('avg_opponent_rating', 0):.0f}")

# Performance by rating difference
rating_diff = opponent.get('by_rating_diff', {})
print("\n📊 Performance by Rating Difference:")
categories = ['much_lower', 'lower', 'similar', 'higher', 'much_higher']
labels = {
    'much_lower': '<<< Much Lower (-200+)',
    'lower': '<< Lower (-100 to -199)',
    'similar': '≈ Similar (±99)',
    'higher': '>> Higher (+100 to +199)',
    'much_higher': '>>> Much Higher (+200+)'
}

for cat in categories:
    data = rating_diff.get(cat, {})
    if data.get('games', 0) > 0:
        print(f"\n  {labels[cat]}:")
        print(f"    Win Rate: {data.get('win_rate', 0):.1f}%")
        print(f"    Games: {data.get('games', 0)} ({data.get('wins', 0)}W-{data.get('losses', 0)}L-{data.get('draws', 0)}D)")

OPPONENT STRENGTH ANALYSIS

Average Opponent Rating: 1825

📊 Performance by Rating Difference:

  ≈ Similar (±99):
    Win Rate: 53.1%
    Games: 81 (43W-37L-1D)

  >> Higher (+100 to +199):
    Win Rate: 0.0%
    Games: 1 (0W-1L-0D)


## Section 7: Time of Day Analysis

Performance based on when games are played.

In [39]:
# Display Time of Day Analysis
time_perf = analysis_results['sections']['time_of_day']

print("=" * 60)
print("TIME OF DAY ANALYSIS")
print("=" * 60)

time_periods = {
    'morning': '🌅 Morning (6am-12pm)',
    'afternoon': '☀️ Afternoon (12pm-6pm)',
    'evening': '🌆 Evening (6pm-10pm)',
    'night': '🌙 Night (10pm-6am)'
}

for period, label in time_periods.items():
    data = time_perf.get(period, {})
    if data.get('games', 0) > 0:
        print(f"\n{label}:")
        print(f"  Win Rate: {data.get('win_rate', 0):.1f}%")
        print(f"  Games: {data.get('games', 0)} ({data.get('wins', 0)}W-{data.get('losses', 0)}L-{data.get('draws', 0)}D)")
        print(f"  Avg Rating: {data.get('avg_rating', 0):.0f}")

# Find best time
best_time = max(time_perf.items(), key=lambda x: x[1].get('win_rate', 0) if isinstance(x[1], dict) else 0)
if best_time and isinstance(best_time[1], dict) and best_time[1].get('games', 0) > 0:
    print(f"\n⭐ Best Performance: {time_periods.get(best_time[0], best_time[0])} ({best_time[1].get('win_rate', 0):.1f}% win rate)")

TIME OF DAY ANALYSIS

🌅 Morning (6am-12pm):
  Win Rate: 47.6%
  Games: 21 (10W-11L-0D)
  Avg Rating: 1832

☀️ Afternoon (12pm-6pm):
  Win Rate: 70.0%
  Games: 10 (7W-3L-0D)
  Avg Rating: 1827

🌆 Evening (6pm-10pm):
  Win Rate: 0.0%
  Games: 2 (0W-2L-0D)
  Avg Rating: 1806

🌙 Night (10pm-6am):
  Win Rate: 53.1%
  Games: 49 (26W-22L-1D)
  Avg Rating: 1828

⭐ Best Performance: ☀️ Afternoon (12pm-6pm) (70.0% win rate)


## Section 8: Mistake Analysis (Milestone 8)

Game stage mistake analysis using Stockfish engine (early game, middle game, endgame).

In [40]:
# Display Mistake Analysis
mistake_analysis = analysis_results['sections'].get('mistake_analysis', {})

print("=" * 60)
print("MISTAKE ANALYSIS (Stockfish)")
print("=" * 60)

# Check if analysis was performed
sample_info = mistake_analysis.get('sample_info', {})
if sample_info.get('analyzed_games', 0) > 0:
    print(f"\nSample Info:")
    print(f"  Total Games: {sample_info.get('total_games', 0)}")
    print(f"  Analyzed Games: {sample_info.get('analyzed_games', 0)}")
    print(f"  Sample %: {sample_info.get('sample_percentage', 0):.1f}%")
    
    # Weakest stage
    print(f"\n⚠️ Weakest Stage: {mistake_analysis.get('weakest_stage', 'N/A')}")
    print(f"   Reason: {mistake_analysis.get('weakest_stage_reason', 'N/A')}")
    
    # Display each stage
    stages = {
        'early': '📘 Early Game (Moves 1-15)',
        'middle': '📗 Middle Game (Moves 16-40)',
        'endgame': '📕 Endgame (Moves 41+)'
    }
    
    for stage, label in stages.items():
        stage_data = mistake_analysis.get(stage, {})
        if stage_data.get('total_moves', 0) > 0:
            print(f"\n{label}:")
            print(f"  Total Moves Analyzed: {stage_data.get('total_moves', 0)}")
            print(f"  Inaccuracies: {stage_data.get('inaccuracies', 0)}")
            print(f"  Mistakes: {stage_data.get('mistakes', 0)}")
            print(f"  Blunders: {stage_data.get('blunders', 0)}")
            print(f"  Missed Opportunities: {stage_data.get('missed_opps', 0)}")
            print(f"  Avg CP Loss: {stage_data.get('avg_cp_loss', 0):.1f}")
            
            # Critical mistake game
            critical = stage_data.get('critical_mistake_game')
            if critical:
                print(f"  Worst Game: Move {critical.get('move_number', 'N/A')} - {critical.get('type', 'N/A')} ({critical.get('cp_loss', 0):.0f} CP loss)")
                print(f"    URL: {critical.get('game_url', 'N/A')}")
else:
    print("\n⚠️ Mistake analysis not available")
    print("   Reasons:")
    print("   - Engine analysis disabled")
    print("   - No games to analyze")
    print("   - Stockfish not installed or not in PATH")

MISTAKE ANALYSIS (Stockfish)

⚠️ Mistake analysis not available
   Reasons:
   - Engine analysis disabled
   - No games to analyze
   - Stockfish not installed or not in PATH


## Section 9: AI Coaching Advice (Milestone 9)

AI-generated personalized coaching recommendations based on the analysis.

**Note:** Requires OpenAI API key to be configured.

In [13]:
# Display AI Coaching Advice
ai_advice = analysis_results['sections'].get('ai_coaching_advice', {})

print("=" * 60)
print("AI COACHING ADVICE")
print("=" * 60)

if ai_advice and ai_advice.get('advice'):
    print("\n🤖 Personalized Recommendations:\n")
    print(ai_advice.get('advice', 'No advice generated'))
    
    # Metadata
    metadata = ai_advice.get('metadata', {})
    if metadata:
        print(f"\n---")
        print(f"Generated: {metadata.get('timestamp', 'N/A')}")
        print(f"Model: {metadata.get('model', 'N/A')}")
        print(f"Tokens: {metadata.get('tokens_used', 'N/A')}")
else:
    print("\n⚠️ AI coaching advice not available")
    print("   Reasons:")
    print("   - OpenAI API key not configured")
    print("   - AI advice generation disabled")
    print("   - Analysis data insufficient")
    print("\nTo enable AI coaching:")
    print("   1. Set OPENAI_API_KEY in .env file")
    print("   2. Set include_ai_advice=True in analysis")

AI COACHING ADVICE

⚠️ AI coaching advice not available
   Reasons:
   - OpenAI API key not configured
   - AI advice generation disabled
   - Analysis data insufficient

To enable AI coaching:
   1. Set OPENAI_API_KEY in .env file
   2. Set include_ai_advice=True in analysis


## Summary: Complete Analysis Results

View the complete JSON structure of all analysis results.

In [14]:
# Display complete analysis structure (abbreviated)
print("=" * 60)
print("COMPLETE ANALYSIS SUMMARY")
print("=" * 60)

print(f"\nTotal Games: {analysis_results.get('total_games', 0)}")
print(f"\nAvailable Sections:")
for section_name in analysis_results.get('sections', {}).keys():
    print(f"  ✓ {section_name}")

# Optional: Save to JSON file
save_json = False  # Set to True to save results

if save_json:
    output_file = f"chess_analysis_{USERNAME}_{START_DATE}_to_{END_DATE}.json"
    with open(output_file, 'w') as f:
        json.dump(analysis_results, f, indent=2, default=str)
    print(f"\n✓ Results saved to: {output_file}")
else:
    print("\n📝 Set save_json=True to export results to JSON file")

COMPLETE ANALYSIS SUMMARY

Total Games: 36

Available Sections:
  ✓ overall_performance
  ✓ color_performance
  ✓ elo_progression
  ✓ termination_wins
  ✓ termination_losses
  ✓ opening_performance
  ✓ opponent_strength
  ✓ time_of_day
  ✓ mistake_analysis

📝 Set save_json=True to export results to JSON file


In [16]:
## Manual check

In [17]:
df_games

,url,game_id,date,time_class,rules,player_color,player_rating,player_result,opponent_username,opponent_rating,opponent_result,rating_diff,pgn,eco,opening,time_control,result,day_of_week,hour
0,https://www.chess.com/game/live/148414876662,148414876662,2026-01-24 14:51:11,blitz,chess,white,1824,win,bin_4hmad,1910,timeout,-86,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/Colle-System-An...,,300,Win,Saturday,14
1,https://www.chess.com/game/live/148416422918,148416422918,2026-01-24 15:45:18,blitz,chess,white,1816,resigned,jacs2409,1830,win,-14,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/Slav-Defense-Mo...,,300,Loss,Saturday,15
2,https://www.chess.com/game/live/148478069120,148478069120,2026-01-26 00:44:03,blitz,chess,white,1823,win,Preusse1,1785,resigned,38,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/Dutch-Defense-C...,,300,Win,Monday,0
3,https://www.chess.com/game/live/148478182772,148478182772,2026-01-26 00:52:31,blitz,chess,black,1815,timeout,vishalpnp1984,1817,win,-2,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/French-Defense-...,,300,Loss,Monday,0
4,https://www.chess.com/game/live/148505216784,148505216784,2026-01-26 17:36:30,blitz,chess,white,1808,timeout,ermarchal,1862,win,-54,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/English-Opening...,,300,Loss,Monday,17
5,https://www.chess.com/game/live/148505635612,148505635612,2026-01-26 17:48:55,blitz,chess,black,1800,resigned,paneenameusoe,1803,win,-3,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/French-Defense-...,,300,Loss,Monday,17
6,https://www.chess.com/game/live/148519596048,148519596048,2026-01-27 00:01:34,blitz,chess,black,1808,win,Ossichesss,1772,timeout,36,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/Kings-Indian-At...,,300,Win,Tuesday,0
7,https://www.chess.com/game/live/148519943482,148519943482,2026-01-27 00:07:28,blitz,chess,white,1816,win,andrew_titov,1819,resigned,-3,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/Slav-Defense-Mo...,,300,Win,Tuesday,0
8,https://www.chess.com/game/live/148520201648,148520201648,2026-01-27 00:13:31,blitz,chess,black,1823,win,Edyp63,1781,resigned,42,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/French-Defense-...,,300,Win,Tuesday,0
9,https://www.chess.com/game/live/148520434378,148520434378,2026-01-27 00:22:22,blitz,chess,white,1830,win,Gahlot,1764,timeout,66,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",https://www.chess.com/openings/Caro-Kann-Defen...,,300,Win,Tuesday,0


In [28]:
# Debug: Check analysis results structure
print("Checking analysis_results structure...")
print(f"\nTotal games: {analysis_results.get('total_games', 'N/A')}")
print(f"\nSections available: {list(analysis_results.get('sections', {}).keys())}")

# Check a few key sections
print("\n" + "="*60)
print("Sample data from key sections:")
print("="*60)

overall = analysis_results['sections'].get('overall_performance', {})
print(f"\n1. Overall Performance:")
print(f"   - win_rate: {overall.get('win_rate', 'N/A')}")
print(f"   - total: {overall.get('total', 'N/A')}")

term_wins = analysis_results['sections'].get('termination_wins', {})
print(f"\n2. Termination Wins:")
print(f"   - total_wins: {term_wins.get('total_wins', 'N/A')}")
print(f"   - breakdown: {term_wins.get('breakdown', 'N/A')}")

openings = analysis_results['sections'].get('opening_performance', {})
print(f"\n3. Opening Performance:")
print(f"   - best_openings: {openings.get('best_openings', 'N/A')}")
print(f"   - worst_openings: {openings.get('worst_openings', 'N/A')}")

print("\n" + "="*60)
print("Checking raw game data...")
print("="*60)
print(f"all_games type: {type(all_games)}")
print(f"all_games length: {len(all_games)}")
if all_games:
    print(f"\nFirst game keys: {list(all_games[0].keys())[:10]}")
    print(f"First game sample:")
    print(f"  - white: {all_games[0].get('white', {}).get('username', 'N/A')}")
    print(f"  - black: {all_games[0].get('black', {}).get('username', 'N/A')}")
    print(f"  - white result: {all_games[0].get('white', {}).get('result', 'N/A')}")
    print(f"  - black result: {all_games[0].get('black', {}).get('result', 'N/A')}")

Checking analysis_results structure...

Total games: 82

Sections available: ['overall_performance', 'color_performance', 'elo_progression', 'termination_wins', 'termination_losses', 'opening_performance', 'opponent_strength', 'time_of_day', 'mistake_analysis']

Sample data from key sections:

1. Overall Performance:
   - win_rate: N/A
   - total: N/A

2. Termination Wins:
   - total_wins: N/A
   - breakdown: N/A

3. Opening Performance:
   - best_openings: N/A
   - worst_openings: N/A

Checking raw game data...
all_games type: <class 'list'>
all_games length: 82

First game keys: ['url', 'pgn', 'time_control', 'end_time', 'rated', 'tcn', 'uuid', 'initial_setup', 'fen', 'time_class']
First game sample:
  - white: Ossichesss
  - black: jay_fh
  - white result: timeout
  - black result: win
